In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import ast
import numpy as np

In [ ]:

# Inicialización de MediaPipe Pose.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2, enable_segmentation=True, min_detection_confidence=0.5)

# Directorio que contiene los videos.
video_directory = 'D:/Codes/EFBA/DataEfba/videos'
video_files = [os.path.join(video_directory, f) for f in os.listdir(video_directory) if os.path.isfile(os.path.join(video_directory, f))]
excel_directory = 'D:/Codes/EFBA/DataEfba/keypoints'

# Procesar cada video en el directorio.
for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    keypoints_list = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Conversión del color del frame de BGR a RGB.
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesamiento del frame para detectar la pose.
        results = pose.process(frame_rgb)
        
        if results.pose_landmarks:
            # Extracción de las coordenadas de los puntos clave.
            keypoints = []
            for landmark in results.pose_landmarks.landmark:
                keypoints.append((landmark.x, landmark.y, landmark.z, landmark.visibility))
            keypoints_list.append(keypoints)

    cap.release()
    
    # Opcional: Convertir la lista de puntos clave a un DataFrame de pandas para un análisis más fácil.
    df_keypoints = pd.DataFrame(keypoints_list, columns=[f'x{i}' for i in range(11)] + [f'y{i}' for i in range(11)] + [f'z{i}' for i in range(11)])
    
    # Aquí puedes guardar o procesar df_keypoints como prefieras.
    # Por ejemplo, guardar en un archivo CSV.
    output_filename = os.path.splitext(os.path.basename(video_file))[0] + '_keypoints.csv'
    df_keypoints.to_csv(os.path.join(excel_directory, output_filename), index=False)

    print(f"Procesado {video_file}, resultados guardados en {output_filename}.")


In [ ]:
def process_column(data):
    try:
        # Usa ast.literal_eval para convertir la cadena a una tupla
        return pd.DataFrame([ast.literal_eval(x) for x in data])
    except ValueError:
        # En caso de error, regresa un DataFrame vacío
        return pd.DataFrame()

def process_csv(file_path):
    df = pd.read_csv(file_path)
    
    # Preparar un diccionario para recopilar las nuevas columnas
    new_data = {}
    
    # Iterar sobre las columnas originales y procesar cada una
    for column in df.columns:
        # Si la columna es de tipo object (cadena), procesar su contenido
        if df[column].dtype == object:
            processed_data = process_column(df[column])
            # Si el procesamiento fue exitoso y el DataFrame no está vacío
            if not processed_data.empty:
                # Crear nuevas columnas para 'x', 'y', 'z' y 'visibility'
                new_data[f'{column}_x'] = processed_data[0]
                new_data[f'{column}_y'] = processed_data[1]
                new_data[f'{column}_z'] = processed_data[2]
                new_data[f'{column}_visibility'] = processed_data[3]
    
    # Crear un nuevo DataFrame con las nuevas columnas
    new_df = pd.DataFrame(new_data)
    
    # Guardar el nuevo DataFrame en un archivo CSV
    output_file_path = os.path.join(output_directory, os.path.basename(file_path).replace('.csv', '_processed.csv'))
    new_df.to_csv(output_file_path, index=False)
    print(f"Archivo procesado y guardado: {output_file_path}")

# Define el directorio donde se encuentran tus archivos CSV
directory_path = 'D:/Codes/EFBA/DataEfba/keypoints'
output_directory = 'D:/Codes/EFBA/DataEfba/pkeypoints'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Lista todos los archivos CSV en el directorio
csv_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.csv')]

# Procesa cada archivo CSV en el directorio
for file in csv_files:
    process_csv(file)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

def normalize_features(file_path, output_directory):
    # Cargar el DataFrame
    df = pd.read_csv(file_path)
    
    # Define las columnas de características que deseas normalizar
    feature_columns = [
        'y0_x', 'y0_y', 'y0_z', 'y0_visibility',
'y1_x', 'y1_y', 'y1_z', 'y1_visibility',
'y2_x', 'y2_y', 'y2_z', 'y2_visibility',
'y3_x', 'y3_y', 'y3_z', 'y3_visibility',
'y4_x', 'y4_y', 'y4_z', 'y4_visibility',
'y5_x', 'y5_y', 'y5_z', 'y5_visibility',
'y6_x', 'y6_y', 'y6_z', 'y6_visibility',
'y7_x', 'y7_y', 'y7_z', 'y7_visibility',
'y8_x', 'y8_y', 'y8_z', 'y8_visibility',
'y9_x', 'y9_y', 'y9_z', 'y9_visibility',
'y10_x', 'y10_y', 'y10_z', 'y10_visibility',
'z0_x', 'z0_y', 'z0_z', 'z0_visibility',
'z1_x', 'z1_y', 'z1_z', 'z1_visibility',
'z2_x', 'z2_y', 'z2_z', 'z2_visibility',
'z3_x', 'z3_y', 'z3_z', 'z3_visibility',
'z4_x', 'z4_y', 'z4_z', 'z4_visibility',
'z5_x', 'z5_y', 'z5_z', 'z5_visibility',
'z6_x', 'z6_y', 'z6_z', 'z6_visibility',
'z7_x', 'z7_y', 'z7_z', 'z7_visibility',
'z8_x', 'z8_y', 'z8_z', 'z8_visibility',
'z9_x', 'z9_y', 'z9_z', 'z9_visibility',
'z10_x', 'z10_y', 'z10_z', 'z10_visibility'
        # Asegúrate de incluir todas tus características aquí
    ]
    
    # Instancia el objeto StandardScaler
    scaler = StandardScaler()
    
    # Aplica la normalización
    df[feature_columns] = scaler.fit_transform(df[feature_columns])
    
    # Construir el path de salida
    output_file_path = os.path.join(output_directory, os.path.basename(file_path).replace('.csv', '_normalized.csv'))
    
    # Guardar el DataFrame normalizado
    df.to_csv(output_file_path, index=False)
    print(f"Archivo normalizado guardado en: {output_file_path}")

# Define la carpeta donde se encuentran tus archivos CSV y la carpeta de salida
input_folder_path = 'D:/Codes/EFBA/DataEfba/pkeypoints/'
output_folder_path = 'D:/Codes/EFBA/DataEfba/nkeypoints/'

# Crea el directorio de salida si no existe
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Obtener una lista de todos los archivos CSV en la carpeta de entrada
csv_files = [os.path.join(input_folder_path, f) for f in os.listdir(input_folder_path) if f.endswith('.csv')]

# Normalizar las características de cada archivo CSV
for file in csv_files:
    normalize_features(file, output_folder_path)


In [ ]:

# Ruta al directorio con los archivos normalizados
normalized_folder_path = 'D:/Codes/EFBA/DataEfba/nkeypoints/'

# Lista todos los archivos en el directorio
files = [os.path.join(normalized_folder_path, f) for f in os.listdir(normalized_folder_path) if f.endswith('_normalized.csv')]

# Carga todos los archivos en un DataFrame
df_features = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)


In [ ]:

# Definir la ruta al directorio con los archivos normalizados
input_directory = 'D:/Codes/EFBA/DataEfba/nkeypoints/'
output_directory = 'D:/Codes/EFBA/DataEfba/nkeypoints_wframe'

# Crear el directorio de salida si no existe
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Definir la tasa de frames
frame_rate = 30  # reemplaza con la tasa de frames real de tus videos

# Lista todos los archivos CSV en el directorio de entrada
files = [f for f in os.listdir(input_directory) if f.endswith('.csv')]

# Procesar cada archivo
for file in files:
    # Cargar el archivo CSV
    df = pd.read_csv(os.path.join(input_directory, file))
    
    # Añadir la columna de frame asumiendo que la primera fila corresponde al frame 0
    df['frame'] = df.index / frame_rate
    
    # Guardar el DataFrame actualizado en el directorio de salida
    df.to_csv(os.path.join(output_directory, file), index=False)

print("Proceso completado. Los archivos actualizados están en:", output_directory)


In [ ]:
import pandas as pd

# Asumiendo que la ruta al archivo es correcta
df_annotations = pd.read_csv('D:/Codes/EFBA/DataEfba/Efba_criterios_temp.csv')

# Verificar las columnas
print("Columnas en df_annotations:", df_annotations.columns)

# Si el nombre de la columna está mal escrito o no como lo esperabas, aquí lo verás


In [ ]:

# Caminos a las carpetas
features_directory = 'D:/Codes/EFBA/DataEfba/nkeypoints_wframe'
annotations_path = 'D:/Codes/EFBA/DataEfba/Efba_criterios_temp.csv'
output_directory = 'D:/Codes/EFBA/DataEfba'

# Cargar las anotaciones
df_annotations = pd.read_csv(annotations_path)

# Expandir las anotaciones a cada frame relevante
expanded_annotations = []
for _, row in df_annotations.iterrows():
    for frame in range(row['frame_inicio'], row['frame_final'] + 1):
        expanded_annotations.append({
            'nombre_del_video': row['nombre_del_video'],
            'frame': frame,
            'criterio': row['criterio'],
            'correcto_o_incorrecto': row['correcto_o_incorrecto'],
            'jugador': row['jugador']
        })

# Convertir la lista expandida de anotaciones a un DataFrame
df_expanded_annotations = pd.DataFrame(expanded_annotations)

# Procesar cada archivo de características
for filename in os.listdir(features_directory):
    if filename.endswith('.csv'):
        df_features = pd.read_csv(os.path.join(features_directory, filename))
        
        # Fusionar los DataFrames basado en 'nombre_del_video' (ajustar si el nombre del archivo es el ID) y 'frame'
        df_merged = pd.merge(df_features, df_expanded_annotations, on=['nombre_del_video', 'frame'], how='left')
        
        # Guardar el DataFrame combinado
        df_merged.to_csv(os.path.join(output_directory, f"{filename[:-4]}_merged.csv"), index=False)

print("Todos los archivos han sido procesados y fusionados.")


In [34]:
# Ruta al directorio con los archivos fusionados
merged_directory = 'D:/Codes/EFBA/DataEfba/mergeddata'

# Directorios para guardar los datos escalados
scaled_output_directory = 'D:/Codes/EFBA/DataEfba/scaleddata'
if not os.path.exists(scaled_output_directory):
    os.makedirs(scaled_output_directory)


In [35]:
for filename in os.listdir(merged_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(merged_directory, filename)
        df = pd.read_csv(file_path)
        
        # Identificar columnas numéricas para escalar
        cols_to_scale = df.select_dtypes(include=[np.number]).columns.tolist()
        cols_non_numeric = df.select_dtypes(exclude=[np.number]).columns.tolist()
        
        # Escalar solo las columnas numéricas
        scaler = StandardScaler()
        df_scaled = pd.DataFrame(scaler.fit_transform(df[cols_to_scale]), columns=cols_to_scale, index=df.index)
        
        # Reintegra las columnas no numéricas
        df_final_scaled = pd.concat([df_scaled, df[cols_non_numeric]], axis=1)
        
        # Guardar el DataFrame escalado
        output_path = os.path.join(scaled_output_directory, filename)
        df_final_scaled.to_csv(output_path, index=False)

        print(f"Procesado y guardado: {output_path}")


C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Procesado y guardado: D:/Codes/EFBA/DataEfba/scaleddata\allen_keypoints_processed_normalized_merged.csv


C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Procesado y guardado: D:/Codes/EFBA/DataEfba/scaleddata\bird_keypoints_processed_normalized_merged.csv


C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Procesado y guardado: D:/Codes/EFBA/DataEfba/scaleddata\curry1_keypoints_processed_normalized_merged.csv


C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\mache12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Procesado y guardado: D:/Codes/EFBA/DataEfba/scaleddata\kerr_keypoints_processed_normalized_merged.csv


In [36]:
from sklearn.model_selection import train_test_split

# Asumiendo que 'df_final_scaled' es tu DataFrame final que incluye tanto características como etiquetas
# Define tus características y etiquetas
X = df_final_scaled.drop(['criterio', 'correcto_o_incorrecto'], axis=1)  # Ajusta esto basado en tus nombres de columnas de etiquetas
y = df_final_scaled[['criterio', 'correcto_o_incorrecto']]  # Ajusta esto según tus necesidades

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Inicializar el clasificador RandomForest
forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Hacer que el modelo soporte la clasificación multietiqueta
model = MultiOutputClassifier(forest, n_jobs=-1)


In [38]:
model.fit(X_train, y_train)

ValueError: Input y contains NaN.